In [1]:
import pandas   as pd
import numpy    as np
import os
from tqdm import tqdm
import json

In [2]:
data_csv_train_fd_path = "/workspace/nabang1010/LBA_VAIPE/GNN/GNN_Drugnames_Extraction_from_Prescription/data_GNN/data_csv_train_gen"
data_csv_train_gen_mapping_fd_mapping = "/workspace/nabang1010/LBA_VAIPE/GNN/GNN_Drugname_Classification_from_Prescription/data_GNN/data_csv_train_gen_mapping"
data_train_json_fd_path = "/workspace/nabang1010/LBA_VAIPE/GNN/GNN_Drugname_Classification_from_Prescription/data_GNN/data_json_train"

list_csv_file = os.listdir(data_csv_train_fd_path)
pbar = tqdm(sorted(list_csv_file))
for csv_file in pbar:
    pbar.set_description("File %s | " % csv_file )
    json_file = csv_file.replace(".csv", ".json")
    json_file_path = os.path.join(data_train_json_fd_path, json_file)
    json_data = json.load(open(json_file_path, "r"))
    csv_write_path = os.path.join(data_csv_train_gen_mapping_fd_mapping, csv_file)
    # print(json_data)
    # print()
    df = pd.read_csv(os.path.join(data_csv_train_fd_path, csv_file))
    # print(df[df["label"] == "drugname"])
    #fill all label diffirent drugname = ""
    df["label"] = df["label"].apply(lambda x: "" if x != "drugname" else x)
    # print(df)
    df_drugname = df[df["label"] == "drugname"]
    # print(df_drugname)
    for row in range(df_drugname.shape[0]):
        drugname_row = df_drugname.iloc[row]
        xmin = drugname_row["xmin"]
        xmax = drugname_row["xmax"]
        ymin = drugname_row["ymin"]
        ymax = drugname_row["ymax"]
        # get index of row
        # df.index[df['BoolCol']]
        index = df.index[(df['xmin'] == xmin) & (df['xmax'] == xmax) & (df['ymin'] == ymin) & (df['ymax'] == ymax)][0]
        # print(index)
        for box_text in json_data:
            if box_text["label"] == "drugname":
                xmin_box = box_text["box"][0]
                xmax_box = box_text["box"][2]
                ymin_box = box_text["box"][1]
                ymax_box = box_text["box"][3]
                if xmin == xmin_box and xmax == xmax_box and ymin == ymin_box and ymax == ymax_box:
                    # df.at[index,:].label = box_text["mapping"]\
                    df.at[index, "label"] = str(box_text["mapping"])
                    # print(df[index,:])
                    break
    
    df.to_csv(csv_write_path, index=False)
    

File VAIPE_P_TRAIN_999.csv | : 100%|██████████| 1173/1173 [00:08<00:00, 134.10it/s]
